In [1]:
class Sentiment:
    NEGETIVE = "NEGETIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

In [12]:
import random

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGETIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_Text(self):
        return [x.text for x in self.reviews]
    
    def get_Sentiment(self):
        return [x.sentiment for x in self.reviews]
        
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGETIVE, self.reviews))
        print(negative[0])
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        print('Positive',len(positive))
        print('Negative',len(negative))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews )

# Load Data

In [13]:
import json

file_path = '../../CSV/Books_small_10000.json'
reviews = []
with open(file_path) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

In [14]:
reviews[5].score

5.0

In [15]:
reviews[5].sentiment

'POSITIVE'

# Prep Data

In [16]:
len(reviews)

10000

In [17]:
from sklearn.model_selection import train_test_split
help(train_test_split)

Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, **options)
    Split arrays or matrices into random train and test subsets
    
    Quick utility that wraps input validation and
    ``next(ShuffleSplit().split(X, y))`` and application to input data
    into a single call for splitting (and optionally subsampling) data in a
    oneliner.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.
    
    test_size : float, int or None, optional (default=0.25)
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        complement of the train size. By default, 

In [38]:
training, test = train_test_split(reviews, test_size = 0.33, random_state = 42)
print(len(training), len(test))

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)


train_container.evenly_distribute()
test_container.evenly_distribute()

6700 3300
Positive 5611
Negative 436
Positive 2767
Negative 208


In [39]:
train_x = train_container.get_Text()
train_y = train_container.get_Sentiment()
test_x = test_container.get_Text()
test_y = test_container.get_Sentiment()

print("POSITIVE",train_y.count(Sentiment.POSITIVE),
"NEGATIVE",train_y.count(Sentiment.NEGETIVE), sep=" ")

POSITIVE 436 NEGATIVE 436


In [40]:
print(train_x[0], train_y[0], sep=' >>  ')

I read this in one sitting- I have always loved Paris, but so much of this resonated. The people. The places, the laughs, the love. There is some great advice for people, enough travelogue to make you pack your bag. Sumptuous moments to savor the food morsels, and a love story to warm your heart. Oh the possibilities. ! >>  POSITIVE


## Bag of Words Vectorization

In [61]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

vectorizer = TfidfVectorizer()
#vectorizer = CountVectorizer()

#1 Step Process
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

# 2 Step Process
# vectorizer.fit(train_x)
# train_x_vector = vectorizer.transform(train_x)

train_x_vectors[0]


<1x8906 sparse matrix of type '<class 'numpy.float64'>'
	with 44 stored elements in Compressed Sparse Row format>

# Classification

### Linear SVM

In [62]:
from sklearn.svm import SVC

clf_svm = SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [63]:
print(test_x[0],
test_y[0])

Better than the previous book but still kinda cheesy and the plot is rather convoluted. It seemed like the Arthur ran out of stuff to say and tried to rush through to the end. The writing is kind of stiff too NEGETIVE


In [64]:
clf_svm.predict(test_x_vectors[0])

array(['NEGETIVE'], dtype='<U8')

### Decision Tree Classifier

In [65]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [66]:
clf_dec.predict(test_x_vectors[0])

array(['NEGETIVE'], dtype='<U8')

### Naive Bayes

In [67]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
#clf_gnb.fit(train_x_vectors, train_y)

#ERROR

### Logistic Regression

In [68]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

/anaconda2/envs/py35/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [69]:
clf_log.predict(test_x_vectors[0])

array(['NEGETIVE'], dtype='<U8')

# Evaluation

### Mean Acuracy

In [70]:
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
#clf_gnb.score(test_x_vectors, test_y)
#Error

print(clf_log.score(test_x_vectors, test_y))

0.8076923076923077
0.6538461538461539
0.8028846153846154


### F1_Score

In [71]:
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors),
         average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGETIVE])

array([0.80582524, 0.80952381])

In [72]:
f1_score(test_y, clf_dec.predict(test_x_vectors),
         average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGETIVE])

/anaconda2/envs/py35/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda2/envs/py35/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


array([0.65048544, 0.        , 0.65714286])

In [73]:
f1_score(test_y, clf_log.predict(test_x_vectors),
         average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGETIVE])

array([0.80097087, 0.        , 0.8047619 ])

In [74]:
train_y[0:5]

['POSITIVE', 'NEGETIVE', 'NEGETIVE', 'POSITIVE', 'POSITIVE']

In [75]:
print('Positive Sentiment : ',train_y.count(Sentiment.POSITIVE))
print('Negative Sentiment : ',train_y.count(Sentiment.NEGETIVE))
print('Neutral Sentiment : ',train_y.count(Sentiment.NEUTRAL))

Positive Sentiment :  436
Negative Sentiment :  436
Neutral Sentiment :  0


In [77]:
test_set = ["I just enjoyed it", "one of the kind", "great look", "do not buy it", "bad great"]
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGETIVE', 'POSITIVE'],
      dtype='<U8')

## Tunning our Model (with Grid Search)

In [82]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel' : ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc = SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

/anaconda2/envs/py35/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'kernel': ('linear', 'rbf'), 'C': (1, 4, 8, 16, 32)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [83]:
print(clf.score(test_x_vectors, test_y))

0.8076923076923077


## Saving Model

### Natural Language Processing

#### Save Model

In [92]:
import pickle

with open('../sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

#### Load Model

In [93]:
with open('../sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [96]:
print(test_x[0])
loaded_clf.predict(test_x_vectors[0])

Better than the previous book but still kinda cheesy and the plot is rather convoluted. It seemed like the Arthur ran out of stuff to say and tried to rush through to the end. The writing is kind of stiff too


array(['NEGETIVE'], dtype='<U8')

In [ ]:
test_set = []
text = input("Enter Sentimental Text")
test_set.append(text)
new_test = vectorizer.transform(test_set)
clf_svm.predict(new_test)